<a href="https://colab.research.google.com/github/ZainulAbideen000748/Machine-Learning-Project-Inventory-/blob/main/Machine_Learning_Project_Final_160126.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 1


BLOCK-1 — Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

sns.set_style('whitegrid')
print("Imports loaded")

Imports loaded


BLOCK-2 — Load Data Sets

In [6]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

df = pd.read_csv('/content/sales_train_validation.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1.0,3.0,0.0,1.0,1.0,1.0,3.0,0.0,1.0,1.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1.0,0.0,5.0,4.0,1.0,0.0,1.0,3.0,7.0,2.0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,4.0


In [7]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

df = pd.read_csv('/content/calendar.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


BLOCK-3 — Melt Wide → Long

In [9]:
sales_df = pd.read_csv('/content/sales_train_validation.csv')
day_cols=[c for c in sales_df.columns if c.startswith('d_')]
long_df=sales_df.melt(id_vars=['id','item_id','store_id'], value_vars=day_cols,
                var_name='day', value_name='daily_sales')
print(long_df)
print(long_df.shape)

                                     id        item_id store_id     day  \
0         HOBBIES_1_001_CA_1_validation  HOBBIES_1_001     CA_1     d_1   
1         HOBBIES_1_002_CA_1_validation  HOBBIES_1_002     CA_1     d_1   
2         HOBBIES_1_003_CA_1_validation  HOBBIES_1_003     CA_1     d_1   
3         HOBBIES_1_004_CA_1_validation  HOBBIES_1_004     CA_1     d_1   
4         HOBBIES_1_005_CA_1_validation  HOBBIES_1_005     CA_1     d_1   
...                                 ...            ...      ...     ...   
21884715    FOODS_3_064_CA_4_validation    FOODS_3_064     CA_4  d_1913   
21884716    FOODS_3_065_CA_4_validation    FOODS_3_065     CA_4  d_1913   
21884717    FOODS_3_066_CA_4_validation    FOODS_3_066     CA_4  d_1913   
21884718    FOODS_3_067_CA_4_validation    FOODS_3_067     CA_4  d_1913   
21884719    FOODS_3_068_CA_4_validation    FOODS_3_068     CA_4  d_1913   

          daily_sales  
0                 0.0  
1                 0.0  
2                 0.0  
3  